In [5]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
import csv
# for parallel processing
from multiprocess import Pool
from scipy.sparse.csgraph import shortest_path

# import plotting functions
from harvey_plotting_v53 import *

# import loop extrusion codes
import pyximport
pyximport.install(setup_args={'include_dirs':np.get_include()})
from loop_extrusion_twoLEFpopulations_CollidingSuperExtruder_SuperLoading_v15 import LEFTranslocatorDirectional

ModuleNotFoundError: No module named 'harvey_plotting_v53'

In [4]:
pip install multiprocess

     |████████████████████████████████| 132 kB 5.0 MB/s eta 0:00:01
     |████████████████████████████████| 110 kB 6.1 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


# Set up chromosome and get all boundary element coordinates

In [ ]:
# define output folder 
output_folder = 'output_1013_test_run1'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

tiling_factor = 600 # number of TAD arrays repeated

# set up boundary elements
boundary_coordinates = [0, 200,201, 601,602, 802,803, 1603,1604, 1804,1805, 2205,2206, 2406,2407, 3607] # coordinates of boundary elements
TAD_array_size =  boundary_coordinates[-1] + 1 # the size of the big TAD in kb
chrom_size = TAD_array_size*tiling_factor # the total size of the chromosome in kb
boundary_coordinates_o = boundary_coordinates # coordinates of boundary elements within the first TAD array

# obtain the coordinates of all boundary elements and the double strand break sites on the chromosome
for j in range(tiling_factor-1):
    k = TAD_array_size*(j+1)
    res = [x + k for x in boundary_coordinates_o] 
    boundary_coordinates = boundary_coordinates + res
    

# DSB coordiantes set up, no need

In [ ]:
# randomly determine the coordinate of the DSB in the first TAD
random_pick_0 = int(np.random.random()*(boundary_coordinates_o[1]-boundary_coordinates_o[0]-2)+boundary_coordinates_o[0]+1)
if random_pick_0+1 == boundary_coordinates[1]:
    DSB_coordinates = [random_pick_0-1, random_pick_0]
else:
    DSB_coordinates = [random_pick_0, random_pick_0+1]

# counters for the number of subTADs with a double strand break
tad_200 = 1 
tad_400 = 0
tad_800 = 0
tad_1200 = 0
tad_count = np.asarray([1,0,0,0]) # counter for TAD of each size containing DSB
tad_size = np.asarray([200, 400, 800, 1200]) dist_between_break = 10000 #kb distances between two DSB

DSB_dist_to_CTCF = [min(DSB_coordinates[0]-boundary_coordinates[0],boundary_coordinates[1]-DSB_coordinates[0])] 

DSB_TAD_array_size = [200]
within_tile = True
while within_tile:
    # skip dist_between_break 
    next_break = DSB_coordinates[-1] + dist_between_break
    # preventing the next DSB occurring at a boundary element
    if np.count_nonzero(next_break - np.asarray(boundary_coordinates))<len(boundary_coordinates):
        next_break += 2
    # determine the TAD the next DSB occurs in
    test = np.asarray(boundary_coordinates) - next_break
    boundray_index = np.argmax(test>0) # the first BE coordinates to the right of the DSB
    # randomize the coordinate of the DSB within the TAD (so that the distance between DSB and BE is randomized)
    random_pick = int(np.random.random()*(boundary_coordinates[boundray_index]-boundary_coordinates[boundray_index-1]-2)+boundary_coordinates[boundray_index-1]+1)
    if random_pick+1 == boundary_coordinates[boundray_index]:
        DSB_coordinates += [random_pick-1, random_pick]
    else:
        DSB_coordinates += [random_pick, random_pick+1]
    tad_size_index = np.argwhere(tad_size==boundary_coordinates[boundray_index]-boundary_coordinates[boundray_index-1])[0][0]
    tad_count[tad_size_index]+=1
    DSB_TAD_array_size.append(tad_size[tad_size_index])
    if DSB_coordinates[-1] + dist_between_break> chrom_size-1:
        within_tile = False


DSB_TAD_array_size = np.asarray(DSB_TAD_array_size)
        
LogFileName = output_folder+"/log.txt"

o = open(LogFileName, "w") 
o.close()

with open(LogFileName, "a") as f:
    f.write('Tiling factor: \n')
    f.write(str(tiling_factor)+'\n')
    for i in range(len(tad_size)):
        f.write('number of breaks within a '+str(tad_size[i])+'kb TADs: \n')
        f.write(str(tad_count[i])+'\n')

file = output_folder+'/DSB_boundary_coordinates.npy' 

# saving DSB coordinates and boundary coordinates to robustness 
if not os.path.exists(file):
    with open(file, 'wb') as g:
        np.save(g, np.asarray(DSB_coordinates))
        np.save(g, np.asarray(boundary_coordinates))

# sweep parameters (from Fbn2)

In [3]:
## Define simulation parameters
# list of (separation,processivity) tuple 
sep_proc_list = []
sep_list = [125]
proc_list = [250]
for sep in sep_list:
    for proc in proc_list:
        sep_proc_list.append((sep,proc))
        
proc_ratio = 20 # the ratio of long-lived LEF processivity over normal LEF processivity  
BE_stabilization_factor_list =  [16] # the fold stabilization of LEF at BE
DSB_stabilization_factor_list = [4] # the fold stabilization of LEF at DSB ends
boundary_strength_list = [0.5] # the probability of LEF stalled by an boundary element
longlived_fraction_list = [0.2] # percentage of long-lived LEFs in the total LEF polulation
targetedloading_factor_list = [1000] # fold increase of loading probability at the DSB ends

alpha_list = [1, 3, 5]   # threshold distance: half of the unfilled gap size (in kb) for calling synapsis

# other input parameters
LEF_step_probability = 1 # the probability LEF taking a step
    
# maximum number of steps post DSB to stop simulation
step_count_limit = 40000

# whether to take snapshots of the synapsis process
plot_snapshots = False

In [ ]:
def do_one_parameter_set(sep_proc_pair, boundary_strength, BEstabilization_factor, 
                         DSBstabilization_factor,longlived_fraction, super_loading_factor, 
                         proc_ratio,chrom_size, step_count_limit,alpha_list,boundary_coordinates,DSB_coordinates,
                         LEF_step_probability,DSB_TAD_array_size, plot_snapshots = False):
    
    # assume uniform loading probability on the whole chromosome
    loading_prob = np.ones(chrom_size)/chrom_size # the probability LEF loads at each lattice site on the chromosome
    stall_prob_left = np.zeros(chrom_size, dtype=np.double) # numpy array to store the probability of left motor subunits of LEFs being stalled
    stall_prob_right = np.zeros(chrom_size, dtype=np.double) # numpy array to store the probability of right motor subunits of LEFs being stalled
    
    separations = sep_proc_pair[0]
    processivity = sep_proc_pair[1]
    processivity_longlived = processivity * proc_ratio #processivity of long-lived LEFs
    
    # if boundary_strength is 0, there is no stabilization of LEF at BE
    BEstabilization_factor_o = BEstabilization_factor 
    if boundary_strength == 0:
        BEstabilization_factor = 1